📝 Notebook Scraping Data Sekolah

Notebook ini digunakan untuk **scraping data sekolah di Indonesia** dari website [SekolahKita](https://sekolah.data.kemendikdasmen.go.id/).  

Tujuan dari notebook ini:  
- Mengumpulkan data sekolah per jenjang (SMA, SMK, MA, dll). Pada kasus ini berfokus pada data SMA.
- Menyimpan hasil scraping per batch untuk menghindari timeout atau error  
- Menggabungkan semua batch menjadi satu file CSV utama  
- Menyediakan data yang siap digunakan untuk **analisis, visualisasi, atau scraping profil sekolah lebih detail** di notebook lanjutan `2_scraping_profil`


# **LIBRARY**

Proyek ini menggunakan beberapa library dari **Python Standard Library**, **Third-party**, dan tools untuk menjalankan proses **paralel** agar scraping lebih efisien.

In [11]:

# ============================
# Standard Library
# ============================
import os               # Operasi sistem (buat folder, path, dll)
import math             # Fungsi matematika (ceil, floor, dll)
import glob             # Operasi file dengan pola tertentu
import pandas as pd     # Manipulasi data tabel (DataFrame)
import re

# ============================
# Third-party Libraries
# ============================
import requests                # HTTP request (scraping data dari web)
from bs4 import BeautifulSoup  # Parsing HTML
from tqdm import tqdm          # Progress bar interaktif

# ============================
# Concurrency Utilities
# ============================
from concurrent.futures import ThreadPoolExecutor, as_completed  # Multi-worker/concurrent

# **FUNGSI**

Di bagian ini, dibuat dua fungsi utama untuk scraping data sekolah:  

1. `extract_data(soup)` → mengekstrak informasi dari HTML, dipakai di dalam fungsi `scrape_pages()`.  
2. `scrape_pages(page_mulai, page_selesai, jenjang, nama_file)` → melakukan scraping per halaman dan menyimpan hasil ke CSV.

In [ ]:
# ============================
# Fungsi Extract Data
# ============================
def extract_data(soup):
    """
    Ekstrak data sekolah dari halaman HTML.
    Mengambil: nama, link profil, alamat, kabupaten, kota, provinsi.
    """
    semua_data = []
    blocks = soup.find_all("div", class_="box box-default")  # setiap sekolah ditampilkan dalam blok HTML

    for block in blocks:
        # Ambil nama sekolah dan link profil
        nama_tag = block.find("a", class_="text-info")
        if not nama_tag:
            continue

        kode_nama = nama_tag.get_text(strip=True)
        kode = re.search(r'\((\d+)\)', kode_nama).group(1)
        nama = re.sub(r'\(\d+\)\s*', '', kode_nama)
        link = nama_tag.get("href", "")

        # Ambil alamat (bisa terdiri dari beberapa elemen <li>)
        alamat_tags = block.find_all("li", class_="list-group-item text-muted")
        alamat_list = [tag.get_text(strip=True) for tag in alamat_tags]
        alamat = " ".join(alamat_list)

        # Inisialisasi wilayah
        kabupaten, kota, provinsi = "", "", ""
        for tag in alamat_list:
            if "Prov." in tag:  # format biasanya "Kab./Kota X Prov. Y"
                bagian = tag.split("Prov.")
                wilayah = bagian[0].strip()
                provinsi = bagian[1].strip()

                if wilayah.startswith("Kab."):
                    kabupaten = wilayah
                elif wilayah.startswith("Kota"):
                    kota = wilayah
                break

        # Simpan hasil
        semua_data.append({
            "kode":kode,
            "nama": nama,
            "link_profil": link,
            "alamat": alamat,
            "kabupaten": kabupaten,
            "kota": kota,
            "provinsi": provinsi
        })

    return semua_data


# ============================
# Fungsi Scrape Pages
# ============================
def scrape_pages(page_mulai, page_selesai, jenjang, nama_file):
    """
    Scrape data sekolah berdasarkan halaman dan jenjang.
    Menyimpan hasil ke CSV.
    """
    url = "https://sekolah.data.kemendikdasmen.go.id/index.php/Chome/pencarian/"
    headers = {"User-Agent": "Mozilla/5.0"}
    semua_data = []

    # Loop tiap halaman
    for page in range(page_mulai, page_selesai + 1):
        payload = {
            "page": page,
            "kode_kabupaten": "",
            "kode_kecamatan": "",
            "bentuk_pendidikan": jenjang,
            "status_sekolah": "semua",
            "nama": ""
        }

        try:
            # Request POST
            response = requests.post(url, data=payload, headers=headers, timeout=15)
            if response.status_code != 200:
                print(f"⚠ Halaman {page} tidak berhasil diakses (status {response.status_code})")
                continue

            # Parsing HTML hasil response
            soup = BeautifulSoup(response.text, "html.parser")
            semua_data.extend(extract_data(soup))

        except Exception as e:
            print(f"[{nama_file}] ❌ Error di halaman {page}: {e}")

    # Simpan ke CSV dengan pandas
    if semua_data:
        df = pd.DataFrame(semua_data)
        df.to_csv(nama_file, index=False, encoding="utf-8")
        print(f"[{nama_file}] ✅ Disimpan {len(df)} baris")
    else:
        print(f"[{nama_file}] ⚠ Tidak ada data disimpan")


# **KONFIGURASI**

Di bagian ini, kita menyiapkan parameter untuk proses scraping.

> Tabel berikut menjelaskan parameter yang digunakan beserta fungsinya:

| Parameter       | Keterangan                                                   | Fungsi                                                                 |
|-----------------|-------------------------------------------------------------|------------------------------------------------------------------------|
| `jenjang`       | Jenjang sekolah (SMA, SMK, MA, dll). Pada kasus ini SMA      | Digunakan sebagai payload request dan nama folder output               |
| `folder`        | Folder output untuk menyimpan CSV                            | Menyimpan hasil scraping agar terorganisir per jenjang                 |
| `page_mulai`    | Halaman awal scraping                                        | Menentukan halaman pertama yang akan di-scrape                          |
| `page_selesai`  | Halaman akhir scraping                                       | Menentukan halaman terakhir yang akan di-scrape                         |
| `batch_size`    | Ukuran batch untuk scraping                                   | Membagi total halaman menjadi batch agar lebih mudah dikelola          |
| `MAX_WORKER`    | Jumlah worker paralel                                        | Menentukan jumlah thread yang dijalankan secara bersamaan untuk efisiensi |

---

> 📌 Catatan `batch_size` dan `MAX_WORKER`

- Misalkan `page_mulai = 1`, `page_selesai = 3745`, dan `batch_size = 100`, maka hasil scraping akan dibagi menjadi **38 file CSV**, masing-masing bernama:  
  - batch_1-100.csv
  - batch_101-200.csv
  - batch_201-300.csv
  - … 
  - batch_3701-380.csv
  
  Informasi jumlah halaman dapat dilihat pada website: [SekolahKita](https://sekolah.data.kemendikdasmen.go.id/)


- **Jumlah worker (`MAX_WORKER`)**:  
  Makin banyak worker → makin banyak request yang dikirim secara bersamaan → lebih cepat, tapi **rawan error** akibat koneksi internet atau limit server.

- **Batch size (`batch_size`)**:  
  - Makin besar batch → tiap file CSV berisi lebih banyak halaman → efisien dalam penyimpanan.  
  - Tapi batch besar juga → proses tiap batch lebih lama dan jika terjadi error, harus ulang batch besar tersebut.  
  - Makin kecil batch → lebih sering menulis file CSV → lebih aman jika ada error, tapi lebih banyak file CSV yang dihasilkan.


In [4]:
# ----------------------------
# Parameter Scraping
# ----------------------------
jenjang = "SMA"
folder = f"../hasil/data_halaman/{jenjang.lower()}"
os.makedirs(folder, exist_ok=True)  # buat folder jika belum ada

page_mulai = 1
page_selesai = 100
batch_size = 50
MAX_WORKER = 2

# Hitung total halaman & batch
total_halaman = page_selesai - page_mulai + 1
total_batch = math.ceil(total_halaman / batch_size)

print(f"Total scraping: {total_halaman} halaman ({total_batch} batch, batch size {batch_size})")


Total scraping: 100 halaman (2 batch, batch size 50)


# **SCRAPPING**

Di bagian ini, kita melakukan scraping data menggunakan fungsi `scrape_pages()` yang sudah dibuat.  
Terdapat dua versi:
1. Tanpa multi-worker → berjalan secara berurutan.
2. Dengan multi-worker → proses paralel lebih cepat.

## **Tanpa Multi-Worker**

In [55]:
print(f"🔄 Mulai scraping {total_halaman} halaman ({total_batch} batch, batch size {batch_size})")

# looping batch secara berurutan (synchronous)
for i in tqdm(range(total_batch), desc="Scraping batch"):
    start = page_mulai + i * batch_size
    end = min(start + batch_size - 1, page_selesai)
    nama_file = f"{folder}/batch_{start}-{end}.csv"

    scrape_pages(start, end, jenjang, nama_file)

print("✅ Semua batch selesai.")

🔄 Mulai scraping 10 halaman (1 batch, batch size 10)


Scraping batch: 100%|██████████| 1/1 [00:50<00:00, 50.64s/it]

[../hasil/data_link/sma/batch_31-40.csv] ✅ Disimpan 40 baris
✅ Semua batch selesai.


## **Dengan Multi-Worker**

In [ ]:
print(f"🔄 Mulai scraping {total_halaman} halaman ({total_batch} batch, batch size {batch_size})")

with ThreadPoolExecutor(max_workers=MAX_WORKER) as executor:
    futures = []
    for i in range(total_batch):
        start = page_mulai + i * batch_size
        end = min(start + batch_size - 1, page_selesai)
        nama_file = f"{folder}/batch_{start}-{end}.csv"

        futures.append(executor.submit(scrape_pages, start, end, jenjang, nama_file))

    # tampilkan progress dengan tqdm
    for _ in tqdm(as_completed(futures), total=total_batch, desc="Scraping batch"):
        pass

print("✅ Semua batch selesai.")

Mengecek Hasil Scrapping

In [9]:
pd.read_csv(f"{folder}/batch_1-100.csv").head()

,kode,nama,link_profil,alamat,kabupaten,kota,provinsi
0,20325309,SMA NU HASYIM ASY ARI,https://sekolah.data.kemdikbud.go.id/index.php...,"JL. RAYA KARANGJATI RT.03/01, TARUB Karangjati...",Kab. Tegal,NaN,Jawa Tengah
1,69913893,SMA NEGERI 1 WULANDONI,https://sekolah.data.kemdikbud.go.id/index.php...,Jl. Trans Wulandoni Wulandoni Kec. Wulandoni K...,Kab. Lembata,NaN,Nusa Tenggara Timur
2,20511951,SMA NEGERI 1 PLOSOKLATEN KABUPATEN KEDIRI,https://sekolah.data.kemdikbud.go.id/index.php...,DS.KAWEDUSAN PLOSOKLATEN Kawedusan Kec. Plosok...,Kab. Kediri,NaN,Jawa Timur
3,20532399,SMAS GALUH HANDAYANI,https://sekolah.data.kemdikbud.go.id/index.php...,Manyar Sambongan 85A Kertajaya Kec. Gubeng Kot...,NaN,Kota Surabaya,Jawa Timur
4,20321976,SMAS THERESIANA WELERI,https://sekolah.data.kemdikbud.go.id/index.php...,JL. TAMTAMA WELERI Penyangkringan Kec. Weleri ...,Kab. Kendal,NaN,Jawa Tengah


## **Menyatukan batch**

Setelah proses scraping per batch selesai, langkah selanjutnya adalah **menggabungkan semua file CSV** menjadi satu file utama.  
Langkah ini memastikan semua data sekolah dari batch berbeda tersimpan dalam satu file `"semua_sma.csv"` (tergantung jenjang) yang siap dianalisis.

In [10]:
# folder hasil batch
csv_files = sorted(glob.glob(os.path.join(folder, "batch_*.csv")))

# cek kalau tidak ada file
if not csv_files:
    raise FileNotFoundError(f"Tidak ada file batch_*.csv di folder {folder}")

print(f"📂 Ditemukan {len(csv_files)} file untuk digabungkan.")

# baca & gabungkan semua file CSV
data_frames = []
for idx, file in enumerate(csv_files, start=1):
    df = pd.read_csv(file)
    print(f"{idx}. {os.path.basename(file)} -> {df.shape[0]} baris")
    data_frames.append(df)

# concat semua DataFrame jadi satu
data_all = pd.concat(data_frames, ignore_index=True)

# simpan hasil gabungan
output_file = f"{folder}/semua_{jenjang.lower()}.csv"
data_all.to_csv(output_file, index=False, encoding="utf-8")

print(f"\n✅ Selesai! Total: {len(data_all)} baris data digabung dan disimpan di: {output_file}")


📂 Ditemukan 38 file untuk digabungkan.
1. batch_1-100.csv -> 400 baris
2. batch_1001-1100.csv -> 400 baris
3. batch_101-200.csv -> 400 baris
4. batch_1101-1200.csv -> 400 baris
5. batch_1201-1300.csv -> 400 baris
6. batch_1301-1400.csv -> 400 baris
7. batch_1401-1500.csv -> 400 baris
8. batch_1501-1600.csv -> 400 baris
9. batch_1601-1700.csv -> 400 baris
10. batch_1701-1800.csv -> 400 baris
11. batch_1801-1900.csv -> 400 baris
12. batch_1901-2000.csv -> 400 baris
13. batch_2001-2100.csv -> 400 baris
14. batch_201-300.csv -> 400 baris
15. batch_2101-2200.csv -> 400 baris
16. batch_2201-2300.csv -> 400 baris
17. batch_2301-2400.csv -> 400 baris
18. batch_2401-2500.csv -> 400 baris
19. batch_2501-2600.csv -> 400 baris
20. batch_2601-2700.csv -> 400 baris
21. batch_2701-2800.csv -> 400 baris
22. batch_2801-2900.csv -> 400 baris
23. batch_2901-3000.csv -> 400 baris
24. batch_3001-3100.csv -> 400 baris
25. batch_301-400.csv -> 400 baris
26. batch_3101-3200.csv -> 400 baris
27. batch_3201-330

✅ Hasil Akhir Scraping

Proses **Scraping Halaman** akan menghasilkan **file CSV** yang berisi data detail **SMA di seluruh Indonesia** sebanyak **`14.927 sekolah`**.  

Kolom pada CSV:

| kode | nama | link_profil | alamat | kabupaten | kota | provinsi |
|------|------|-------------|--------|-----------|------|----------|

File CSV disimpan di folder output sesuai jenjang, misalnya: <br>
`../hasil/data_halaman/sma/semua_sma.csv`

File CSV ini siap digunakan untuk **scraping data sekolah lebih detail**, yang dilakukan pada notebook: <br>
`2_scraping_profil.ipynb`


> Dengan file ini, seluruh proses scraping profil sekolah bisa dilakukan secara otomatis dan sistematis, sehingga data siap dipakai untuk analisis lebih lanjut.